# Colab-VSGAN

My repo: [styler00dollar/VSGAN-tensorrt-docker](https://github.com/styler00dollar/VSGAN-tensorrt-docker/)


The purpose of this notebook is to compile several dependencies manually which  can be used in the inference colab. This colab is not for inference.

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
# check python version, for me its currently 3.8.10
!python --version

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
#@title COMPILING

################################
# mmediting
################################
%cd /content
!git clone https://github.com/open-mmlab/mmediting.git && cd mmediting && pip install -v -e . && python setup.py bdist_wheel

################################
# mmcv
################################
%cd /content
!git clone https://github.com/open-mmlab/mmcv --recursive && cd mmcv && MMCV_WITH_OPS=1 MAKEFLAGS="-j$(nproc)" python setup.py build_ext && MMCV_WITH_OPS=1 python setup.py bdist_wheel

################################
# cupy
################################
%cd /content
!git clone https://github.com/cupy/cupy
%cd cupy
!git submodule update --init && pip install . --verbose && MAKEFLAGS="-j$(nproc)" python setup.py bdist_wheel --verbose

################################
# TensorRT 8.6.1.6-1+cuda12.0
################################
%cd /content
!sudo apt-get update
!sudo apt-get install libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python3-libnvinfer tensorrt python3-libnvinfer-dev

################################
# compile vs
################################
%cd /content
# installing vapoursynth related stuff
!apt install autoconf libtool yasm python3.10 python3.10-venv python3.10-dev ffmsindex libffms2-5 libffms2-dev -y
!apt --fix-broken install
# zimg
!wget https://github.com/sekrit-twc/zimg/archive/refs/tags/release-3.0.4.zip && 7z x release-3.0.4.zip
%cd zimg-release-3.0.4
!./autogen.sh && ./configure && make -j$(nproc) && make install
# vapoursynth
!pip install Cython -U --force-reinstall
%cd /content
!git clone https://github.com/vapoursynth/vapoursynth
%cd vapoursynth
!./autogen.sh && ./configure && make -j$(nproc) && make install && cd .. && ldconfig
!ln -s /usr/local/lib/python3.10/site-packages/vapoursynth.so /usr/lib/python3.10/lib-dynload/vapoursynth.so
#!pip install vapoursynth
!python setup.py bdist_wheel

################################
# install g++11
################################
!sudo apt install build-essential manpages-dev software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt update -y && sudo apt install gcc-11 g++-11 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-11 11
!sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-11 11

################################
# compile vs-mlrt
################################
# updating cmake
%cd /content
!wget https://github.com/Kitware/CMake/releases/download/v3.24.0-rc1/cmake-3.24.0-rc1-linux-x86_64.sh
!chmod +x cmake-3.24.0-rc1-linux-x86_64.sh
!sh cmake-3.24.0-rc1-linux-x86_64.sh --skip-license
!cp /content/bin/cmake /usr/bin/cmake
!cp /content/bin/cmake /usr/lib/x86_64-linux-gnu/cmake
!cp /content/bin/cmake /usr/local/bin/cmake
!cp -r /content/share/cmake-3.24 /usr/local/share/
# getting vapoursynth sourcecode
%cd /content
!wget https://github.com/vapoursynth/vapoursynth/archive/refs/tags/R63.zip
!7z x R63.zip
# compiling
%cd /content/
!sudo rm -rf vs-mlrt
!git clone https://github.com/AmusementClub/vs-mlrt
%cd vs-mlrt/vstrt
!mkdir build
%cd build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 cmake .. -DVAPOURSYNTH_INCLUDE_DIRECTORY=/content/vapoursynth-R63/include
!make -j$(nproc)
!sudo make install
%cd /content

################################
# VMAF
################################
%cd /content
!apt install nasm -y
!pip install ninja meson

!rm -rf vmaf-2.3.1
!wget https://github.com/Netflix/vmaf/archive/refs/tags/v2.3.1.tar.gz
# VMAF
!tar -xzf  v2.3.1.tar.gz
%cd vmaf-2.3.1/libvmaf/
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build --buildtype release
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

%cd /content
!rm -rf VapourSynth-VMAF
!git clone https://github.com/HomeOfVapourSynthEvolution/VapourSynth-VMAF
%cd VapourSynth-VMAF
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

%cd /content
!apt install nasm fftw3-dev -y
!pip install ninja meson

################################
# Vapoursynth-VFRToCFR
################################
!sudo rm -rf Vapoursynth-VFRToCFR
!git clone https://github.com/Irrational-Encoding-Wizardry/Vapoursynth-VFRToCFR && cd Vapoursynth-VFRToCFR && \
    mkdir build && cd build && meson --buildtype release .. && ninja && ninja install

################################
# vapoursynth-mvtools
################################
%cd /content
!sudo rm -rf vapoursynth-mvtools
!git clone https://github.com/dubhater/vapoursynth-mvtools && cd vapoursynth-mvtools && ./autogen.sh && ./configure && make -j$(nproc) && make install

################################
# fmtconv
################################
%cd /content
!sudo rm -rf fmtconv
!git clone https://github.com/EleonoreMizo/fmtconv && cd fmtconv/build/unix/ && ./autogen.sh && ./configure && make -j$(nproc) && make install

################################
# MISC
################################
!sudo apt-get install -y libvulkan-dev nasm
!pip install meson ninja

%cd /content
!rm -rf vs-miscfilters-obsolete
!git clone https://github.com/vapoursynth/vs-miscfilters-obsolete
%cd vs-miscfilters-obsolete
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

################################
# RIFE
################################
%cd /content
!rm -rf VapourSynth-RIFE-ncnn-Vulkan
!git clone https://github.com/styler00dollar/VapourSynth-RIFE-ncnn-Vulkan
%cd VapourSynth-RIFE-ncnn-Vulkan
!git submodule update --init --recursive --depth 1
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 meson build
!CC=/usr/bin/gcc-11 CXX=/usr/bin/g++-11 ninja -C build && ninja -C build install

################################
# Vapoursynth-VFRToCFR
################################
%cd /content
!git clone https://github.com/Irrational-Encoding-Wizardry/Vapoursynth-VFRToCFR && cd Vapoursynth-VFRToCFR && \
    mkdir build && cd build && meson --buildtype release .. && ninja && ninja install

################################
# vapoursynth-mvtools
################################
%cd /content
!git clone https://github.com/dubhater/vapoursynth-mvtools && cd vapoursynth-mvtools && ./autogen.sh && ./configure && make -j$(nproc) && make install

################################
# akarin vs
################################
%cd /content
!apt install llvm-12 llvm-12-dev -y && git clone https://github.com/AkarinVS/vapoursynth-plugin && \
    cd vapoursynth-plugin && meson build && ninja -C build && \
    ninja -C build install

################################
# scxvid
################################
%cd /content
!apt install libxvidcore-dev -y && apt-get autoclean -y && apt-get autoremove -y && apt-get clean -y && \
    git clone https://github.com/dubhater/vapoursynth-scxvid && cd vapoursynth-scxvid && ./autogen.sh && ./configure && make -j$(nproc) && make install

################################
# wwxd
################################
%cd /content
!git clone https://github.com/dubhater/vapoursynth-wwxd && cd vapoursynth-wwxd && \
    gcc -o libwwxd.so -fPIC -shared -O2 -Wall -Wextra -Wno-unused-parameter $(pkg-config --cflags vapoursynth) src/wwxd.c src/detection.c && \
    cp libwwxd.so /usr/local/lib/libwwxd.so

################################
# dfttest2
################################
%cd /content
!git clone https://github.com/AmusementClub/vs-dfttest2 --recursive && cd vs-dfttest2 && cmake -S . -B build -D ENABLE_CUDA=ON -D ENABLE_CPU=ON && cmake --build build && \
    cmake --install build

################################
# lsmash
################################
%cd /content
!git clone https://code.videolan.org/videolan/dav1d/ && \
  cd dav1d && meson build --buildtype release -Ddefault_library=static && ninja -C build install
!git clone https://github.com/FFmpeg/FFmpeg && cd FFmpeg && \
    CFLAGS=-fPIC ./configure --enable-shared --disable-static --enable-pic && make -j$(nproc) && make install && ldconfig && cd /content && rm -rf FFmpeg && \
    git clone https://github.com/l-smash/l-smash && cd l-smash && CFLAGS=-fPIC ./configure --disable-static --enable-shared  && make -j$(nproc) && make install && cd /content && rm -rf l-smash && \
    git clone https://github.com/AkarinVS/L-SMASH-Works && cd L-SMASH-Works && git switch ffmpeg-4.5 && cd VapourSynth/ && meson build && ninja -C build && ninja -C build install

################################
# julek
################################
%cd /content
!apt install clang libstdc++-12-dev -y
!git clone https://github.com/dnjulek/vapoursynth-julek-plugin --recurse-submodules -j8 && cd vapoursynth-julek-plugin/thirdparty && mkdir libjxl_build && cd libjxl_build && \
    cmake -C ../libjxl_cache.cmake -DCMAKE_CXX_COMPILER=clang++ -DCMAKE_C_COMPILER=clang -G Ninja ../libjxl && cmake --build . && cmake --install . && cd ../.. && \
    cmake -DCMAKE_CXX_COMPILER=clang++ -B build -DCMAKE_BUILD_TYPE=Release -G Ninja && cmake --build build && cmake --install build

################################
# warpsharp
################################
%cd /content
!git clone https://github.com/dubhater/vapoursynth-awarpsharp2 && cd vapoursynth-awarpsharp2 && mkdir build && cd build && meson ../ && ninja && ninja install && \
    cd /workspace

################################
# descale
################################
%cd /content
!git clone https://github.com/Irrational-Encoding-Wizardry/descale && cd descale && meson build && ninja -C build && ninja -C build install

################################
# bestsource
################################
!apt-get install libjansson-dev -y && git clone https://github.com/vapoursynth/bestsource && cd bestsource && git clone https://github.com/sekrit-twc/libp2p.git --depth 1 && \
  meson build && ninja -C build && ninja -C build install

In [ ]:
#@title copy files
%cd /content
!sudo rm -rf compiled
!mkdir compiled
%cd compiled

# /usr/local/lib/libwwxd.so
!cp /content/vapoursynth-wwxd/libwwxd.so /content/compiled

# /usr/local/lib/x86_64-linux-gnu/libawarpsharp2.so
!cp /content/vapoursynth-awarpsharp2/build/libawarpsharp2.so /content/compiled

# /usr/local/lib/libfmtconv.so
!cp /content/fmtconv/build/unix/.libs/libfmtconv.so /content/compiled

# /usr/local/lib/vapoursynth/libakarin.so
!cp /content/vapoursynth-plugin/build/libakarin.so /content/compiled

# /usr/local/lib/vapoursynth/libjulek.so
!cp /content/vapoursynth-julek-plugin/build/libjulek.so /content/compiled

# /usr/local/lib/vapoursynth/librife.so
!cp /content/VapourSynth-RIFE-ncnn-Vulkan/build/librife.so /content/compiled

# /usr/local/lib/libmvtools.so
!cp /content/vapoursynth-mvtools/.libs/libmvtools.so /content/compiled

# /usr/local/lib/libscxvid.so
!cp /content/vapoursynth-scxvid/.libs/libscxvid.so /content/compiled

# /usr/local/lib/vapoursynth/libdescale.so
!cp /content/descale/build/libdescale.so /content/compiled

# /usr/local/lib/x86_64-linux-gnu/vapoursynth/libvfrtocfr.so
!cp /content/Vapoursynth-VFRToCFR/build/libvfrtocfr.so /content/compiled

# /usr/local/lib/libvapoursynth.so
!cp /content/vapoursynth/.libs/libvapoursynth.so /content/compiled

# /usr/local/lib/libvapoursynth-script.so
!cp /content/vapoursynth/.libs/libvapoursynth-script.so /content/compiled

# /usr/lib/python3.10/lib-dynload/vapoursynth.so
# /usr/local/lib/python3.10/site-packages/vapoursynth.so
#!cp /content/zimg-release-3.0.4/vapoursynth/.libs/vapoursynth.so /content/compiled
!cp /usr/lib/python3.10/lib-dynload/vapoursynth.so /content/compiled/vapoursynth_dyn.so
!cp /usr/local/lib/python3.10/site-packages/vapoursynth.so /content/compiled

# /usr/local/lib/libzimg.so
!cp /content/zimg-release-3.0.4/.libs/libzimg.so /content/compiled

# /usr/local/lib/libvstrt.so
!cp /content/vs-mlrt/vstrt/build/libvstrt.so /content/compiled

# /usr/local/lib/vapoursynth/libmiscfilters.so
!cp /content/vs-miscfilters-obsolete/build/libmiscfilters.so /content/compiled

# /usr/local/lib/vapoursynth/libvslsmashsource.so
!cp /content/L-SMASH-Works/VapourSynth/build/libvslsmashsource.so /content/compiled

# /usr/local/lib/vapoursynth/libvmaf.so
!cp /content/VapourSynth-VMAF/build/libvmaf.so /content/compiled/libvmaf_vs.so

# /usr/local/lib/x86_64-linux-gnu/libvmaf.so
!cp /content/vmaf-2.3.1/libvmaf/build/src/libvmaf.so /content/compiled

# /usr/local/lib/vapoursynth/libbestsource.so
!cp /usr/local/lib/vapoursynth/libbestsource.so /content/compiled

!cp /content/vapoursynth/.libs/vspipe /content/compiled

!cp /content/vapoursynth/dist/*.whl /content/compiled
!cp /content/mmcv/dist/*.whl /content/compiled
!cp /content/cupy/dist/*.whl /content/compiled
!cp /content/mmediting/dist/*.whl /content/compiled

!7z a compiled.7z *

!cp compiled.7z /content/drive/MyDrive/